# Pandas Project - Alberto Ibarra

### Importar librerias para la limpieza y Cargar base de datos

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df_raw_data = pd.read_csv('attacks.csv', encoding='latin-1')

In [3]:
print('Registros iniciales:',df_raw_data.shape[0])

Registros iniciales: 25723


In [4]:
df_raw_data.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [5]:
df_raw_data = df_raw_data.rename(columns= {'Case Number':'Case','Sex ':'Sex','Species ':'Species','Fatal (Y/N)':'Fatal','Investigator or Source':'Source'})

# Limpieza inicial de filas

### Quitar filas duplicadas

In [6]:
df_lessdup_data=df_raw_data.copy()
df_lessdup_data=df_lessdup_data.drop_duplicates()
print('Filas descartadas por duplicacion:',df_raw_data.shape[0]-df_lessdup_data.shape[0])
print('Registros actuales:',df_lessdup_data.shape[0])

Filas descartadas por duplicacion: 19411
Registros actuales: 6312


### Detección de cantidad de datos invalidos

In [7]:
Datosvalidos = df_lessdup_data.isnull().sum()
Datosvalidos

Case                 2
Date                10
Year                12
Type                14
Country             60
Area               465
Location           550
Activity           554
Name               220
Sex                575
Age               2841
Injury              38
Fatal              549
Time              3364
Species           2848
Source              27
pdf                 10
href formula        11
href                10
Case Number.1       10
Case Number.2       10
original order       3
Unnamed: 22       6311
Unnamed: 23       6310
dtype: int64

### Se detrmina como datos muy relevantes: caso, fecha, año, tipo, pais y daño, por lo que las filas que no cuenten con alguno de estos datos serán descartadas

In [8]:
df_row_cleaned=df_lessdup_data.dropna(subset=['Case','Date','Year','Type','Country','Injury'])

In [9]:
print('Filas descartadas:', df_lessdup_data.shape[0] - df_row_cleaned.shape[0])
print('Registros Actuales:', df_row_cleaned.shape[0])

Filas descartadas: 93
Registros Actuales: 6219


In [10]:
Datosvalidos = df_row_cleaned.isnull().sum()
#Datosvalidos

## Asignacion de valores a todos los datos no validos

In [11]:
c_fill_ukn = {'Area':'UNKNOWN','Location':'UNKNOWN','Activity':'UNKNOWN','Name':'UNKNOWN','Sex': '?','Age':0,'Fatal':'?','Time':'?','Species':'UNKNOWN','Source':'UNKNOWN','href formula':'UNKNOWN','Unnamed: 22':0,'Unnamed: 23':0}
df_row_cleaned.fillna(c_fill_ukn,inplace=True,downcast=True)
#df_row_cleaned.isnull().sum()

/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


# Limpieza de columnas

## Limpieza columna Case

In [12]:
#Se les dará un numero de caso único segun su aparicion en la lista comenzando del final al principio comenzando con 
#las letras 'SA' - Shark Atack y rellenando con ceros las 5 primeras posiciones

df_col01_clean = df_row_cleaned.copy()
df_col01_clean.reset_index(drop=True, inplace=True)

for x in range(len(df_col01_clean['Case'])):
    nceros= 5 - len(str(len(df_col01_clean['Case']) - x))
    ceros = ''
    for i in range(nceros):
        ceros += '0'
    df_col01_clean.iloc[x,0] = 'SA'+ ceros +str(len(df_col01_clean['Case']) - x)

In [13]:
#display(df_col01_clean.head(2))
#display(df_col01_clean.tail(2))

## Limpieza columna Date

In [14]:
df_col02_clean = df_col01_clean.copy()

Logica de la limpieza:
hacer lista de meses para comparacion
buscar con la lista de meses y extraerlos en una columna nueva
quitar todas las palabras que aparezcan antes del primero numero
extraer los dos  o un primeros numeros y asignarlo como dia en una nueva columna
quitar dias y meses 

-Revisar
Buscar años que quedan y borrarlos hasta 1900
Buscar registros sin año y asignar el que viene en la fecha
Asignar "UNKNOWN" a los registros que no tengan indicio de la fecha




In [15]:
#Funcion general para buscar en una cadena varios elementos en una lista y regresar el elemento encontrado
#o regresar una cadena configurable

#Recibe x- elemento a comprobar, 
#keylist - lista de comparacion, 
#strrep - cadena de caracteres para asignar a los elementos no encontrados y 
#repwith: cierto para reemplazar
#replace - cierto reemplaza con strrep los elementos no encontrados, falso solo reemplaza lo encontrado
#secref - cierto para recibir una segunda referencia para asignar
#lstref - lista de referencia a asignar - debe tener la misma longitud de elementos de keylist

#Return cadena de caracteres en la lista o cadena de excepcion al no encontrarla
def isinlist(x,keylist,strrep = '',repwith = False,replace = True,secref=False,listref=[]): 
    for i in range(len(keylist)):
        if x.find(keylist[i])>=0:
            if secref==False:
                return keylist[i]
            else:
                return listref[i]
    if replace == True:
        if repwith == True:
            return strrep + x
        elif repwith == False:
            return strrep
    elif replace == False:
        return x

In [16]:
#Funcion para extraer dia mes(en formato MMM) y año de una cadena
def exdmy(x,sel='date',hem='N'):
    numeros = re.findall(r'\d+',x) #Pone en una lista las secuencias numericas completas 
    salida = [0,'mes',-2000] #Salida 0 = dia, Salida 1 = año - Inicializacion
    salida[1] = isinlist(x,months,'UNKNOWN')
    
    for i in range(len(numeros)):
        nint = int(numeros[i])
        if nint>0 and nint<32 and salida[0] == 0: # El primer numero entre 1 y 31 se asigna como dia
            salida[0] = nint
        elif nint >32 and nint<2021: # Los numeros entre 32 y 2021 se asignan como año
            salida[2] = nint
        elif nint==-2000: # si no ha sido asignado un año y existe un numeros  mayor de -2000 que se encuentre, se asigna como año
            salida[2] = nint
    if salida[0]==0:
        salida[0]='UNKNOWN'
    if salida[2]==-2000:
        salida[2]='UNKNOWN'
    
    #Calculo de dia del año
    if salida[0] != 'UNKNOWN' and salida[0]>0 and salida[1]!='UNKNOWN':
        dict_nmo = {'JAN':1, 'FEB':2,'MAR':3,'APR':4,'MAY':5,'JUN':6,'JUL':7,'AUG':8,'SEP':9,'OCT':10,'NOV':11,'DEC':12} #Meses
        dinmo = [31,28,31,30,31,30,31,31,30,31,30,31] 
        nmo=dict_nmo[salida[1]]
        diny=salida[0]
        if nmo>1:
            for i in range(nmo-1):
                diny+=dinmo[i]
        if salida[2]!='UNKNOWN' and salida[2]%4==0: #Año bisiesto
            diny+=1
    else:
        diny="UNKNOWN"
    
    #Estacion: Regresa una lista con dos elementos indicando la estacion en el hemisferio norte y en el hemisferio sur
    season ="UNKNOWN"
    if diny =='UNKNOWN':
        season = {'N':'UNKNOWN','S':'UNKNOWN'}
    elif diny !='UNKNOWN' and diny >=80 and diny <=171:
        season = {'N':'SPRING','S':'AUTUMN'}
    elif diny !='UNKNOWN' and diny >= 172 and diny <=264:
        season = {'N':'SUMMER','S':'WINTER'}
    elif diny !='UNKNOWN' and diny >=265 and diny <=354:
        season = {'N':'AUTUMN','S':'SPRING'}
    else:
        season = {'N':'WINTER','S':'SUMMER'}
    

    if sel == 'date':
        return salida
    if sel == 'd':
        return salida[0]
    if sel == 'm':
        return salida[1]
    if sel == 'y':
        return salida[2]
    if sel == 'season':
        return season[hem]
    if sel == 'diny':
        return diny

In [17]:
#Extraccion dia mes y año
months=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC']
df_col02_clean['Date'] = df_col02_clean['Date'].apply(lambda x: x.upper()) #mayusculas
df_col02_clean['Day'] = df_col02_clean['Date'].apply(lambda x: exdmy(x,'d')) #extraccion del dia, mes y año y creacion de 3 nuevas columna
df_col02_clean['Month'] = df_col02_clean['Date'].apply(lambda x: exdmy(x,'m')) #extraccion del dia, mes y año y creacion de 3 nuevas columna
df_col02_clean['Year2'] = df_col02_clean['Date'].apply(lambda x: exdmy(x,'y')) #extraccion del dia, mes y año y creacion de 3 nuevas columna
df_col02_clean['Season'] = df_col02_clean['Date'].apply(lambda x: exdmy(x,'season')) #extraccion del dia, mes y año y creacion de 3 nuevas columna

for i in range(len(df_col02_clean['Date'])):
    if str(df_col02_clean['Year2'][i]).isnumeric() and df_col02_clean['Month'][i]!='UNKNOWN' and str(df_col02_clean['Day'][i]).isnumeric():
        df_col02_clean['Date'][i]= str(df_col02_clean['Day'][i])+ '-' + df_col02_clean['Month'][i] + '-' + str(df_col02_clean['Year2'][i])
    elif df_col02_clean['Month'][i]!='UNKNOWN' and str(df_col02_clean['Day'][i]).isnumeric():
        df_col02_clean['Date'][i]= str(df_col02_clean['Day'][i])+ '-' + df_col02_clean['Month'][i]
    else:
        df_col02_clean['Date'][i]= 'UNKNOWN'

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


## Limpieza columna Year

In [18]:
df_col03_clean= df_col02_clean.copy(deep=True)
df_col03_clean['Year']=df_col03_clean['Year'].apply(lambda x: int(x))
#df_col03_clean['Year'].value_counts()
df_col03_clean['Year2'] = df_col03_clean['Year2'].apply(lambda x: int(0) if x=="UNKNOWN" else x)

#Descarte de los datos del año 0 al año 200
"""
indexNames = df_col03_clean[df_col03_clean['Year']<500]

# delete all rows with column 'Age' has value 30 to 40 
indexNames = dfObj[ (dfObj['Age'] >= 30) & (dfObj['Age'] <= 40) ].index
dfObj.drop(indexNames , inplace=True)
# delete all rows with column 'Age' has value 30 to 40 
indexNames = dfObj[ (dfObj['Age'] >= 30) & (dfObj['Age'] <= 40) ].index
dfObj.drop(indexNames , inplace=True)
"""
#Descarte de columnas con variaciones en el año

df_col03_clean['CheckYear'] = abs(df_col03_clean['Year'] - df_col03_clean['Year2'])
pd.options.mode.chained_assignment = None 
indexNames = df_col03_clean[df_col03_clean['CheckYear']!=0].index
print(len(indexNames))

df_col03_clean.drop(indexNames,inplace = True)

print('Registros Actuales:',df_col03_clean.shape[0])


152
Registros Actuales: 6067


#WTF con esto!!??
for i in range(len(df_col03_clean['CheckYear'])):
    if df_col03_clean['CheckYear'][i] >50:
        df_col03_clean['Year'][i] = df_col03_clean['Year2'][i]
#tampoco funciono
df_col03_clean['Year'][indexNames].replace(df_col03_clean['Year2'][indexNames], inplace=True)

In [19]:
indexFechasRaras=df_col03_clean[df_col03_clean['CheckYear']>0].index
df_col03_clean.iloc[indexFechasRaras,:]

,Case,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23,Day,Month,Year2,Season,CheckYear


In [20]:
df_col03_clean['CheckYear'].value_counts()

0    6067
Name: CheckYear, dtype: int64

## Limpieza columna Type

In [21]:
df_col04_clean=df_col03_clean.copy()

In [22]:
df_col04_clean['Type'].value_counts()

Unprovoked      4423
Provoked         560
Invalid          524
Sea Disaster     225
Boating          199
Boat             133
Questionable       2
Boatomg            1
Name: Type, dtype: int64

In [23]:
df_col04_clean['Type']=df_col04_clean['Type'].apply(lambda x:x.upper())
df_col04_clean['Type']=df_col04_clean['Type'].apply(lambda x:'BOATING' if x=='BOAT' or x=='BOATOMG'else x)
df_col04_clean['Type']=df_col04_clean['Type'].apply(lambda x:'UNKNOWN' if x=='INVALID' or x=='QUESTIONABLE' else x)

In [24]:
df_col04_clean['Type'].value_counts()

UNPROVOKED      4423
PROVOKED         560
UNKNOWN          526
BOATING          333
SEA DISASTER     225
Name: Type, dtype: int64

## Limpieza columna Country

In [25]:
series_country = df_col04_clean['Country'].copy()

In [26]:
#Limpieza de la base de datos de paises
df_world = pd.read_csv('countries.csv')
series_countries_valid = df_world[['Country']].copy()  #Copia de la serie
series_countries_valid['Country'] = series_countries_valid['Country'].apply(lambda x: x.upper()) #Mayusculas
series_countries_valid['Country'] = series_countries_valid['Country'].apply(lambda x: x.replace(' ','')) #Espacios
series_countries_valid['Country'] = series_countries_valid['Country'].apply(lambda x: x.replace(',','')) #Comas
series_countries_valid['Country'] = series_countries_valid['Country'].apply(lambda x: x.replace('BAHAMASTHE','BAHAMAS')) #Comas
#series_countries_valid['Country'] = series_countries_valid['Country'].apply(lambda x: re.sub(r"^\s+|\s+$",'',x)) #Quita espacios al principio y al final
list_countries_valid = list(series_countries_valid['Country'])

In [27]:
#list_countries_valid

In [28]:
#Funcion para buscar en la cadena dada un de los posibles paises, si hay dos paises tomará el primero en caso de
#no encontrarlo regresara el mismo dato con una "X-" para decidir el siguiente criterio de limpieza
def iscountry(x):
    for i in range(len(list_countries_valid)):
        if x.find(list_countries_valid[i])>=0:
            return list_countries_valid[i]
        else:
            pass
    return 'X-' + x

In [29]:
#Limpieza de Series ·Countries·
series_country = series_country.apply(lambda x: str(x)) #Transformacion a str para evitar error de tipo de datos
series_country = series_country.apply(lambda x: x.upper()) #Hologacion a mayusculas
series_country = series_country.apply(lambda x: x.replace(' ','')) #Quitar espacios
series_country = series_country.apply(lambda x: x.replace('USA','UNITEDSTATES')) #Excepcion pais con dos nombres
series_country = series_country.apply(lambda x: x.replace('ENGLAND','UNITEDKINGDOM')) #Excepcion pais con dos nombres
series_country = series_country.apply(lambda x: iscountry(x))#Aplicacion de la funcion iscountry que homologa el nomnre de los paises

In [30]:
#Asignacion de la limpieza a la tabla general
df_col05_clean=df_col04_clean.copy()
df_col05_clean['Country']=series_country

In [31]:
b1=series_country.value_counts()
lst_b1=[]
reg_count=0
for x in range(len(b1)):
    if b1.index[x].find('X-')==0:
        lst_b1.append((b1.index[x],b1[x]))
        reg_count+= b1[x] 
#print(lst_b1)
print('Casos con pais mal clasificado:',reg_count)
print('Clasificaciones:',len(df_col05_clean['Country'].value_counts()))


Casos con pais mal clasificado: 178
Clasificaciones: 176


## Limpieza columna Area

In [32]:
df_col06_clean=df_col05_clean.copy()
print('Clasificaciones:',len(df_col06_clean['Area'].value_counts()))

Clasificaciones: 787


## Limpieza columna Location

In [33]:
df_col07_clean=df_col06_clean.copy()
print('Clasificaciones:',len(df_col07_clean['Location'].value_counts()))

Clasificaciones: 4005


## Limpieza columna Activity

In [34]:
df_col08_clean=df_col07_clean.copy()

print('Clasificaciones:',len(df_col08_clean['Activity'].value_counts()))
#df_col08_clean['Activity'].value_counts()   

Clasificaciones: 1471


In [35]:
list_act=['FISHING', 'SURFING', 'DIVING', 'BATHING','WADING', 'STANDING','SNORKEL', 'BODYBOARD', 'FELL',] 

df_col08_clean['Activity'] = df_col08_clean['Activity'].apply(lambda x: x.replace(' ',''))
df_col08_clean['Activity'] = df_col08_clean['Activity'].apply(lambda x: x.upper())
df_col08_clean['Activity'] = df_col08_clean['Activity'].apply(lambda x: isinlist(x,list_act,'UNCLASSIFIED'))

print('Clasificaciones:',len(df_col08_clean['Activity'].value_counts()))
df_col08_clean['Activity'].value_counts()                                                       

Clasificaciones: 10


UNCLASSIFIED    2595
FISHING         1138
SURFING         1117
DIVING           493
BATHING          183
WADING           165
STANDING         126
SNORKEL           98
FELL              85
BODYBOARD         67
Name: Activity, dtype: int64

## Limpieza columna Sex

In [36]:
df_col09_clean=df_col08_clean.copy()
print('Clasificaciones:',len(df_col09_clean['Sex'].value_counts()))
df_col09_clean['Sex'].value_counts()

Clasificaciones: 7


M      4908
F       615
?       539
M         2
lli       1
.         1
N         1
Name: Sex, dtype: int64

In [37]:
df_col09_clean['Sex'] = df_col09_clean['Sex'].apply(lambda x: re.sub(r'\s','',x))
df_col09_clean['Sex'] = df_col09_clean['Sex'].apply(lambda x: x.upper())
df_col09_clean['Sex'] = df_col09_clean['Sex'].apply(lambda x:'?' if x!='M' and x!='F' else x)
df_col09_clean['Sex'].value_counts()

M    4910
F     615
?     542
Name: Sex, dtype: int64

## Limpieza columna Age

In [38]:
df_col10_clean=df_col09_clean.copy()
print('Clasificaciones:',len(df_col10_clean['Age'].value_counts()))
#df_col10_clean['Age'].value_counts()

Clasificaciones: 156


In [39]:
df_col10_clean['Age'] = df_col10_clean['Age'].apply(lambda x: str(x)) #todo a str
df_col10_clean['Age'] = df_col10_clean['Age'].apply(lambda x: re.sub('[^A-Za-z0-9]+','',x)) #quitar caracteres especiales
df_col10_clean['Age'] = df_col10_clean['Age'].apply(lambda x: '?' if x=='0' or x=='' or x==None  else x) #0 años = desconocida
df_col10_clean['Age'] = df_col10_clean['Age'].apply(lambda x: x.upper()) #todo a mayus
df_col10_clean['Age'] = df_col10_clean['Age'].apply(lambda x: x.replace(' ','')) #quitar espacios

age_class = ['CHILD','TEEN','YOUNG ADULT','ADULT','SENIOR']#Clasificacion de eadades
age_ranges=[[1,12],[13,17],[18,29],[30,49],[50,100]]
print('Clasificaciones:',len(df_col10_clean['Age'].value_counts()))
#df_col10_clean['Age'].value_counts()
#df_col10_clean['Age'].head()

Clasificaciones: 137


In [40]:
def age_classificator(x):
    x = str(x)
    if x.isnumeric():
        x= int(x)
        for i in range(len(age_ranges)):
            if x>=age_ranges[i][0] and x<=age_ranges[i][1]:
                return age_class[i]
    else:
        return '?'

In [41]:
df_col10_clean['Age'] = df_col10_clean['Age'].apply(lambda x: age_classificator(x)) #primera clasificacion
print('Clasificaciones:',len(df_col10_clean['Age'].value_counts()))
df_col10_clean['Age'].value_counts()

Clasificaciones: 6


?              2713
YOUNG ADULT    1269
ADULT           876
TEEN            617
SENIOR          297
CHILD           277
Name: Age, dtype: int64

## Limpieza columna Injury

In [42]:
df_col11_clean=df_col10_clean.copy()
print('Clasificaciones:',len(df_col11_clean['Injury'].value_counts()))
#df_col1_clean['Injury'].value_counts()

Clasificaciones: 3649


In [43]:
list_inj=['FATAL', 'SURVIVED', 'BITTEN', 'NOINJURY','MINOR', 'LACERAT'] 

df_col11_clean['Injury'] = df_col11_clean['Injury'].apply(lambda x: x.replace(' ',''))
df_col11_clean['Injury'] = df_col11_clean['Injury'].apply(lambda x: x.upper())
df_col11_clean['Injury'] = df_col11_clean['Injury'].apply(lambda x: isinlist(x,list_inj,'UNCLASSIFIED'))

print('Clasificaciones:',len(df_col11_clean['Injury'].value_counts()))
df_col11_clean['Injury'].value_counts()    

Clasificaciones: 7


BITTEN          1325
FATAL           1267
UNCLASSIFIED    1215
LACERAT         1198
NOINJURY         677
MINOR            233
SURVIVED         152
Name: Injury, dtype: int64

## Limpieza columna Fatal

In [44]:
df_col12_clean=df_col11_clean.copy()
print('Clasificaciones:',len(df_col12_clean['Fatal'].value_counts()))
df_col12_clean['Fatal'].value_counts()

Clasificaciones: 9


N          4196
Y          1294
?           515
UNKNOWN      51
 N            7
M             1
y             1
N             1
2017          1
Name: Fatal, dtype: int64

In [45]:
df_col12_clean['Fatal'] = df_col12_clean['Fatal'].apply(lambda x: x.replace(' ',''))
df_col12_clean['Fatal'] = df_col12_clean['Fatal'].apply(lambda x: x.upper())
df_col12_clean['Fatal'] = df_col12_clean['Fatal'].apply(lambda x:'?' if x!='Y' and x!='N' else x)
df_col12_clean['Fatal'].value_counts()

N    4204
Y    1295
?     568
Name: Fatal, dtype: int64

## Limpieza columna Time

In [46]:
df_col13_clean=df_col12_clean.copy()
print('Clasificaciones:',len(df_col13_clean['Time'].value_counts()))
#df_col13_clean['Time'].value_counts()

Clasificaciones: 365


Logica de la limpieza:

limpiar numeros solo la hora
-Revisar que inicia en numero
-extraer numeros hasta letra
-regresar un solo numero
asignar clasificacion a numeros
clasificar palabras

In [47]:
#homologación básica
list_time=[] 
df_col13_clean['Time'] = df_col13_clean['Time'].apply(lambda x: str(x)) #todo a str
df_col13_clean['Time'] = df_col13_clean['Time'].apply(lambda x: re.sub('[^A-Za-z0-9]+','',x)) #quitar caracteres especiales
df_col13_clean['Time'] = df_col13_clean['Time'].apply(lambda x: x.replace(' ',''))#Quitar Espacios
df_col13_clean['Time'] = df_col13_clean['Time'].apply(lambda x: x.upper())#Todo a mayusculas

print('Clasificaciones:',len(df_col13_clean['Time'].value_counts()))
#df_col13_clean['Time'].value_counts()


Clasificaciones: 335


In [48]:
#Cambiar palabras con aparicion por una hora
filter_time_txt=['AFTERNOON','MORNING','NIGHT','EVENING','LATEAFTERNOON','DUSK','AM','EARLYMORNING','MIDDAY','EARLYAFTERNOON','SUNSET','MIDNIGHT','DAWN',]
filter_time_num=['13','06','22','17','18','19','07','01','12','13','17','00','18']

df_col13_clean['Time'] = df_col13_clean['Time'].apply(lambda x: isinlist(x,filter_time_txt,replace=False,secref=True,listref=filter_time_num))
print('Clasificaciones:',len(df_col13_clean['Time'].value_counts()))
#df_col13_clean['Time'].value_counts()

Clasificaciones: 313


In [49]:
def typefromrange(x,tipos,rangos,nullrange='UNKNOWN'):
    x = str(x)
    if x.isnumeric():
        x= int(x)
        for i in range(len(rangos)):
            if x>=rangos[i][0] and x<=rangos[i][1]:
                return tipos[i]
    else:
        return nullrange

In [50]:
time_ranges=[[0,5],[6,11],[12,18],[19,23]]
time_types=['EARLY','MORNING','AFTERNOON','EVENING','NIGHT']

df_col13_clean['Time'] = df_col13_clean['Time'].apply(lambda x: re.sub('[^0-9]',' ',x)) #buscar los numeros inciales
df_col13_clean['Time'] = df_col13_clean['Time'].apply(lambda x: x.replace(' ','')) #buscar los numeros inciales
df_col13_clean['Time'] = df_col13_clean['Time'].apply(lambda x: '0'+x if len(x) ==1 else x) #poner 0 a elemntos con 1 solo digito
df_col13_clean['Time'] = df_col13_clean['Time'].apply(lambda x: x[0]+x[1] if len(x) > 2 else x)
df_col13_clean['Time'] = df_col13_clean['Time'].apply(lambda x: '99' if x=='' else x)
df_col13_clean['Time'] = df_col13_clean['Time'].apply(lambda x: 'UNKNOWN' if int(x) > 24 else x)
df_col13_clean['Time'] = df_col13_clean['Time'].apply(lambda x: typefromrange(x,time_types,time_ranges))

In [51]:
#df_col13_clean['Time'] = df_col13_clean['Time'].apply(lambda x: hour_cl(x)) #filtro de numeros
#df_col13_clean['Time'] = df_col13_clean['Time'].apply(lambda x: time_class(x)) #clasificacion categorica
#df_col11_clean['Time'] = df_col11_clean['Time'].apply(lambda x: isinlist(x,list_time,'?'))

In [52]:

print('Clasificaciones:',len(df_col13_clean['Time'].value_counts()))
df_col13_clean['Time'].value_counts()

Clasificaciones: 5


UNKNOWN      3187
AFTERNOON    1726
MORNING       926
EVENING       185
EARLY          43
Name: Time, dtype: int64

## Limpieza columna Species

In [53]:
df_col14_clean=df_col13_clean.copy()
print('Clasificaciones:',len(df_col14_clean['Species'].value_counts()))
#df_col14_clean['Species'].value_counts()

Clasificaciones: 1526


In [54]:
#Carga y limpieza de base de datos con nombre de especies de tiburones para comparar
df_shark_types = pd.read_csv('shark_types.csv')
df_shark_types=df_shark_types.drop_duplicates()
series_shark_types=df_shark_types['English']
series_shark_types=series_shark_types.dropna()
series_shark_types=series_shark_types.apply(lambda x: str(x))
series_shark_types=series_shark_types.apply(lambda x: x.upper())
series_shark_types=series_shark_types.apply(lambda x: x.replace('SHARK',''))
series_shark_types=series_shark_types.apply(lambda x: x.replace("'",''))
series_shark_types=series_shark_types.apply(lambda x: re.sub(r'\s', '', x))
series_shark_types=series_shark_types.drop_duplicates()
list_shark_species = list(series_shark_types)

In [55]:
#Funcion para buscar en la cadena dada un de los posibles tipos de tiburones
def isshark(x):
    for i in range(len(list_shark_species)):
        if x.find(list_shark_species[i])>=0:
            return list_shark_species[i]
        else:
            pass
    return 'UNKNOWN'

In [56]:
#Limpieza de Series ·Species·
series_species = df_col14_clean['Species']
series_species = series_species.fillna('UNKNOWN') #Relleno de NA por la palabra UNKNOWN para eliminar Nulos
series_species = series_species.apply(lambda x: str(x)) #Transformacion a str para evitar error de tipo de datos
series_species = series_species.apply(lambda x: x.replace(' ',''))#Eliminacion de espacios
series_species = series_species.apply(lambda x: x.upper()) #Hologacion a mayusculas
series_species = series_species.apply(lambda x: x.replace('SHARK','')) #Eliminacion de la palabra "Shark"
series_species = series_species.apply(lambda x: x.replace('"','')) #Eliminacion de comillas dobles
series_species = series_species.apply(lambda x: isshark(x)) #Aplicacion de la funcion isshark

In [57]:
nasty_col_species = series_species.unique()
print('Especies identificadas: ', nasty_col_species.shape[0]-1)
#series_species.value_counts()

Especies identificadas:  53


In [58]:
#Integracion de la Series ·Species· al data frame
df_col14_clean['Species'] = series_species

In [59]:
print('Clasificaciones:',len(df_col14_clean['Species'].value_counts()))
df_col14_clean['Species'].value_counts()

Clasificaciones: 54


UNKNOWN                4466
WHITE                   624
TIGER                   249
BULL                    179
BLACKTIP                 75
BRONZEWHALER             68
NURSE                    55
BLUE                     51
SPINNER                  48
GREYNURSE                38
LEMON                    33
OCEANICWHITETIP          26
SANDTIGER                21
CARIBBEANREEF            16
DUSKY                    13
WHALE                     9
PORBEAGLE                 8
CARPET                    7
BASKING                   6
ANGEL                     6
COPPER                    5
SANDBAR                   5
GALAPAGOS                 5
THRESHER                  4
SHORTFINMAKO              4
SILKY                     4
ZAMBEZI                   3
BLACK                     3
SALMON                    3
GOBLIN                    2
SCHOOL                    2
DOGFISH                   2
COW                       2
COOKIECUTTER              2
CAT                       2
BROADNOSESEVENGILL  

In [60]:
df_col14_clean.columns

Index(['Case', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal', 'Time', 'Species',
       'Source', 'pdf', 'href formula', 'href', 'Case Number.1',
       'Case Number.2', 'original order', 'Unnamed: 22', 'Unnamed: 23', 'Day',
       'Month', 'Year2', 'Season', 'CheckYear'],
      dtype='object')

In [61]:
cu=['Case', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location','Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal', 'Time', 'Species','Season']
df_col14_clean[cu].head(10)
df_col14_clean[cu].head(10)

for i in range(len(cu)):
    print('Clasificaciones en columna',cu[i],len(df_col14_clean[cu[i]].value_counts()))


Clasificaciones en columna Case 6067
Clasificaciones en columna Date 4679
Clasificaciones en columna Year 235
Clasificaciones en columna Type 5
Clasificaciones en columna Country 176
Clasificaciones en columna Area 787
Clasificaciones en columna Location 4005
Clasificaciones en columna Activity 10
Clasificaciones en columna Name 5082
Clasificaciones en columna Sex 3
Clasificaciones en columna Age 6
Clasificaciones en columna Injury 7
Clasificaciones en columna Fatal 3
Clasificaciones en columna Time 5
Clasificaciones en columna Species 54
Clasificaciones en columna Season 5


In [62]:
df_col14_clean.to_csv('clean.csv')

In [63]:
print(df_col14_clean.head())


      Case         Date  Year        Type       Country             Area  \
0  SA06219  25-JUN-2018  2018     BOATING  UNITEDSTATES       California   
1  SA06218  18-JUN-2018  2018  UNPROVOKED  UNITEDSTATES          Georgia   
2  SA06217   9-JUN-2018  2018     UNKNOWN  UNITEDSTATES           Hawaii   
3  SA06216   8-JUN-2018  2018  UNPROVOKED     AUSTRALIA  New South Wales   
4  SA06215   4-JUN-2018  2018    PROVOKED        MEXICO           Colima   

                         Location      Activity             Name Sex  ...  \
0     Oceanside, San Diego County  UNCLASSIFIED      Julie Wolfe   F  ...   
1  St. Simon Island, Glynn County      STANDING  Adyson McNeely    F  ...   
2                    Habush, Oahu       SURFING      John Denges   M  ...   
3              Arrawarra Headland       SURFING             male   M  ...   
4                        La Ticla        DIVING   Gustavo Ramos    M  ...   

  Case Number.1 Case Number.2 original order Unnamed: 22 Unnamed: 23 Day  \
0   

In [64]:
df_col14_clean.tail()


,Case,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23,Day,Month,Year2,Season,CheckYear
6190,SA00029,UNKNOWN,0,SEA DISASTER,GUINEA,Madang Province,Off Lae,UNCLASSIFIED,male,M,...,ND.0031,ND.0031,31.0,0,0,UNKNOWN,UNKNOWN,0,UNKNOWN,0
6193,SA00026,UNKNOWN,0,UNPROVOKED,GREECE,Dodecanese Islands,Symi Island,DIVING,Psarofa-gomenes,M,...,ND.0027,ND.0027,28.0,0,0,UNKNOWN,UNKNOWN,0,UNKNOWN,0
6196,SA00023,UNKNOWN,0,UNPROVOKED,SOUTHAFRICA,Western Cape Province,Arniston,WADING,Madelaine Dalton,F,...,ND.0023,ND.0023,24.0,0,0,UNKNOWN,UNKNOWN,0,UNKNOWN,0
6197,SA00022,UNKNOWN,0,UNPROVOKED,AUSTRALIA,UNKNOWN,UNKNOWN,DIVING,Jaringoorli,M,...,ND.0022,ND.0022,23.0,0,0,UNKNOWN,UNKNOWN,0,UNKNOWN,0
6198,SA00021,UNKNOWN,0,UNPROVOKED,SOUTHAFRICA,KwaZulu-Natal,Durban,UNCLASSIFIED,Indian boy,M,...,ND.0021,ND.0021,22.0,0,0,UNKNOWN,UNKNOWN,0,UNKNOWN,0
